# Online Continual Learning with Hyperneurons


In [6]:
import numpy as np
import torch

In [5]:
np.exp(1j * 2 * np.pi * np.random.random((4, 5)))

array([[ 0.99961837-0.02762467j, -0.86971774+0.49354944j,
         0.75151489+0.65971613j,  0.95095527+0.30932841j,
        -0.4959941 +0.86832589j],
       [ 0.32682378+0.9450853j ,  0.96356361-0.26747929j,
         0.89689106+0.44225155j,  0.83859536+0.54475482j,
        -0.83306499-0.55317512j],
       [ 0.90827324-0.41837748j, -0.40748979+0.91320976j,
        -0.90103325-0.43375003j, -0.89990769+0.43608043j,
        -0.60192027+0.79855619j],
       [ 0.93147842-0.36379659j, -0.79370852+0.60829827j,
        -0.79125092+0.6114916j ,  0.6123965 +0.79055077j,
         0.04626029+0.99892942j]])

In [7]:
def softmax(x):
    '''Helper function to compute softmax in a numerically stable way.'''
    max_x = np.max(x)
    return np.exp(x - max_x) / np.sum(np.exp(x - max_x))

In [28]:
def lazy_bubblesort(x, order):
    '''
    Update the ordering (order) of a list (x) using a single upward and downward pass of bubblesort.
    Order should be decreasing to represent rankings, while list elements remain in place.
    Single-pass, lazy bubblesort gives a simple way to do incremental sorting over repeated timesteps in O(n).
    '''
    N = len(x)

    # Push smaller elements further down the rankings.
    for i in range(N - 1):
        # Swap orders if the value at the current rank is smaller than that at the next rank.
        if x[order[i]] < x[order[i + 1]]:
                order[i], order[i + 1] = order[i + 1], order[i]

    # # Pull larger elements further up the rankings.
    # for i in range(N - 1):
    #     # Swap orders if the value at the current rank is larger than that at the previous rank.
    #     if x[order[-i]] > x[order[-i - 1]]:
    #         order[-i], order[-i - 1] = order[-i - 1], order[-i]

    # Order of list should be stored by whatever called this so incremental sorting can continue on the next timestep.
    return order

In [29]:
x = np.random.random_sample(10)
x

array([0.64506513, 0.82762656, 0.23912974, 0.63396832, 0.75119008,
       0.16947031, 0.5745722 , 0.35525883, 0.47287316, 0.55571657])

In [30]:
order = np.arange(10)
order

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [33]:
order = lazy_bubblesort(x, order)
order

array([1, 0, 4, 3, 6, 7, 8, 9, 2, 5])

In [32]:
x[order]

array([0.82762656, 0.64506513, 0.63396832, 0.75119008, 0.23912974,
       0.5745722 , 0.35525883, 0.47287316, 0.55571657, 0.16947031])

In [ ]:
# Define a dense associate memory (DAM) that stores complex-valued hyperdimensional vectors and learn online.
class DAM:
    def __init__(self, N, D):
        # N prototypes, D dimensions, scale random correlations to have unit normal similarity distribution.
        self.N = N
        self.D = D
        self.scale = np.sqrt(2. / D)

        # N prototype codes sit on the D-dimensional unit complex hypertorus.
        self.prototypes = np.exp(1j * 2 * np.pi * np.random.random((N, D)))

        #
        self.outputcode = np.exp(1j * 2 * np.pi * np.random.random((D, N)))

        # Track prototype usage frequencies and rankings for Zipf homeostatic regularization.
        self.frequency = 1. / N * np.ones((N, 1))
        self.rankorder = np.arange(1, N + 1)

    def recall(self, query):
        # Find real-valued similarity scores for the query against every prototype in memory,
        # scaled so that random complex hypervectors will have unit normal similarity.
        similarity = self.scale * np.real(np.matmul(np.conj(self.prototypes), query))

        # Apply softmax nonlinearity
        assignment = softmax(self.inv_temp * similarity)

        # Track